# Kelvin-Helmholtz Instability

We will represent an incompressible, viscous, diffusive, Boussinesq, stratified fluid to simulate a Kelvin-Helmholtz instability.

<img src="./Stratified_KH_small.jpeg" width="600" height="300" />



This exercise was designed for the course Waves and Instabilities in Geophysical Fluid Dynamics of the Master's Degree in Advanced Physics and Applied Mathematics, at University of the Balearic Islands (Spain).

Author: Daniel Argüeso
Email: d.argueso@uib.es

Feb-2023

In [ ]:
## Import modules
import numpy as np
from dedalus import public as de
from dedalus.extras import flow_tools
import matplotlib.pyplot as plt
import h5py
import time

: 

In [ ]:
## Import and set logging

: 

In [ ]:
import logging
root = logging.root
for h in root.handlers:
    h.setLevel("INFO")
logger = logging.getLogger(__name__)

: 

In [ ]:
## Define the problem

: 

In [ ]:
### Set problem domain

: 

In [ ]:
Lx, Ly = (2., 1.)
nx, ny = (512, 256)

: 

In [ ]:
### Create bases and domain

: 

In [ ]:
x_basis = de.Fourier('x', nx, interval = (0,Lx), dealias =3/2)
y_basis = de.Chebyshev('y', ny, interval=(-Ly/2, Ly/2), dealias=3/2)
domain = de.Domain([x_basis,y_basis], grid_dtype=np.float64)

: 

In [ ]:
### Set parameters

: 

In [ ]:
Prandtl = 1
Reynolds = 2e4
g = 9.81

: 

In [ ]:
### Define the problem and the equations

: 

In [ ]:
problem = de.IVP(domain, variables = ['p','u','uy','v','vy','rho'])


: 

In [ ]:
problem.parameters['Re'] = Reynolds
problem.parameters['g'] = g

: 

These are the problem equations:

**Equations**

See CR 14.2 (PDF version pg 429 - equations pg 431)

Next we will define the equations that will be solved on this domain.  The equations are

$$ \partial_t u + \boldsymbol{u}\boldsymbol{\cdot}\boldsymbol{\nabla} u + \frac{\partial_x p}{\rho_0} =  \frac{1}{{\rm Re}} \nabla^2 u $$
$$ \partial_t v + \boldsymbol{u}\boldsymbol{\cdot}\boldsymbol{\nabla} v + \frac{\partial_y p}{\rho_0} + \frac{\rho g}{\rho_0} =  \frac{1}{{\rm Re}} \nabla^2 v $$
$$ \boldsymbol{\nabla}\boldsymbol{\cdot}\boldsymbol{u} = 0 $$
$$ \partial_t \rho + \boldsymbol{u}\boldsymbol{\cdot}\boldsymbol{\nabla} \rho = \frac{1}{{\rm RePr}} \nabla^2 \rho $$

The equations are written such that the left-hand side (LHS) is treated implicitly, and the right-hand side (RHS) is treated explicitly.  The LHS is limited to only linear terms, though linear terms can also be placed on the RHS.  Since $y$ is our special direction in this example, we also restrict the LHS to be at most first order in derivatives with respect to $y$.

In [ ]:
problem.add_equation("dt(u) + dx(p) - 1/Re*(dx(dx(u)) + dy(uy)) = - u*dx(u) - v*dy(u)")
problem.add_equation("dt(v) + dy(p) - 1/Re*(dx(dx(v)) + dy(vy)) + g*rho = -u*dx(v) - v*vy")
problem.add_equation("dx(u) + vy = 0")
problem.add_equation("dt(rho) = -u*dx(rho) - v*dy(rho)")
problem.add_equation("vy - dy(v) = 0")
problem.add_equation("uy - dy(u) = 0")

: 

In [ ]:
### Define the boundary conditions

: 

In [ ]:
problem.add_bc("left(u) = 1.0")
problem.add_bc("right(u) = -1.0")
problem.add_bc("left(v) = 0")
problem.add_bc("right(v) = 0", condition="(nx != 0)")
problem.add_bc("integ(p,'y') = 0", condition="(nx == 0)")

: 

In [ ]:
## Define the solver

: 

In [ ]:
### Timestepping

: 

In [ ]:
ts = de.timesteppers.RK443

: 

In [ ]:
### Building the solver

: 

In [ ]:
solver =  problem.build_solver(ts)

x = domain.grid(0)
y = domain.grid(1)
u = solver.state['u']
uy = solver.state['uy']
v = solver.state['v']
vy = solver.state['vy']
p = solver.state['p']
rho = solver.state['rho']

: 

Set the solver parameters

In [ ]:
solver.stop_sim_time = 10.01
solver.stop_wall_time = np.inf
solver.stop_iteration = np.inf

: 

Set initial timestep and CFL conditions

In [ ]:
initial_dt = 0.1*Lx/nx
cfl = flow_tools.CFL(solver,initial_dt,safety=0.5,threshold=0.05)
cfl.add_velocities(('u','v'))

: 

### Initial conditions

In [ ]:
Set initial conditions with a sinusoidal perturbation in vertical velocity

: 

In [ ]:
a = 0.02
amp = -0.2
sigma = 0.2
flow = -1.0
u['g'] = flow*np.tanh(y/a)
rho['g'] = -0.1*np.tanh(y/a)
v['g'] = amp*np.exp(-y**2/sigma**2)*np.sin(4*np.pi*x/Lx)

: 

In [ ]:
## Solving

: 

In [ ]:
Prepare the variables that will be saved for analysis (this is optional)

: 

In [ ]:
analysis = solver.evaluator.add_file_handler('analysis_tasks', sim_dt=0.1, max_writes=50)
analysis.add_task('rho')
analysis.add_task('u')
analysis.add_task('0.5*(u**2+v**2)',name='KE',scales=(3/2,3/2))
solver.evaluator.vars['Lx'] = Lx

: 

In [ ]:
### Plotting initial state

: 

In [ ]:
Make the plot for the initial state

: 

In [ ]:
x = domain.grid(0,scales=domain.dealias)
y = domain.grid(1,scales=domain.dealias)
xm, ym = np.meshgrid(x,y)
fig, axis = plt.subplots(figsize=(8,5))
rho.set_scales(domain.dealias)
u.set_scales(domain.dealias)
v.set_scales(domain.dealias)
p = axis.pcolormesh(xm, ym, rho['g'].T, cmap='RdYlBu')
q = axis.quiver(xm[::10,::10],ym[::10,::10], u['g'][::10,::10].T, v['g'][::10,::10].T)
axis.set_xlim([0,2.])
axis.set_ylim([-0.5,0.5])
plt.savefig(f'./Kelvin-Helmholtz_000.png')

: 

In [ ]:
### Move into solving loop

: 

In [ ]:
logger.info('Starting loop')
start_time = time.time()
nt=1
while solver.ok:
    dt = cfl.compute_dt()
    solver.step(dt)
    if solver.iteration % 10 == 0:
        # Update plot of scalar field and the velocities
        p.set_array(rho['g'].T)
        q.set_UVC(u['g'][::10,::10].T, v['g'][::10,::10].T)
        axis.set_title('t = %f' %solver.sim_time)
        axis.set_title('Density')
        fig.canvas.draw()
        plt.savefig(f'./Kelvin-Helmholtz_{nt:03d}.png')
        nt+=1

: 

### ending program with information

In [ ]:
end_time = time.time()
logger.info('Run time: %f' %(end_time-start_time))
logger.info('Iterations: %i' %solver.iteration)

: 